In [41]:
import polars as pl
from functools import partial
from itertools import product
import multiprocessing
from numba import jit

In [42]:
#Get dataframe of all tickers
constituents = pl.read_csv("../Data/SP500/Constituents.csv", infer_schema_length=False)

#Create list of individual stocks from names from dataframe
stockComponents=[]
for k in range(len(constituents)):
    stockComponents.append(constituents[k,0])
    #stockComponents.append(constituents[k,0]+'.L')

In [43]:
#Load prices and timestamps from CSVs
openPrices=[]
closePrices=[]
timestamps=[]
for k in range(len(stockComponents)):
    currentDf=pl.read_csv("../Data/SP500/dailyHist2000/"+stockComponents[k]+".csv", infer_schema_length=None)
    openPrices.append(currentDf['Open'].to_numpy())
    closePrices.append(currentDf['Close'].to_numpy())
    timestamps.append(currentDf['timestamp'].to_numpy())

In [44]:
@jit(nopython=True)
def algo(paramsList, openPrices, closePrices, timestamps):

    change=paramsList[0]
    takeProfit=paramsList[1]
    stopLoss=paramsList[2]
    timeout=paramsList[3]
    daysBefore=paramsList[4]
    fee=paramsList[5]
    
    tradePercent=0

    total=0
    totalTime=0

    for stock in range(len(openPrices)):
        start=daysBefore
        end=int(len(openPrices[stock])-((timeout/86400)+1))

        for j in range(start, end):
            beforeTime=0
            newTime=j-daysBefore
            while beforeTime<(timestamps[stock][j]-daysBefore*86400):
                beforeTime=timestamps[stock][newTime]
                newTime+=1

            if openPrices[stock][j]<openPrices[stock][j-daysBefore]*(2-change):
                total+=1
                buyPrice = openPrices[stock][j]

                startTime=timestamps[stock][j]

                time=0
                ratio=1

                datapoint=j+1
                while time < timeout and stopLoss<ratio<takeProfit:
                    time = int(timestamps[stock][datapoint])-startTime
                    sellPrice=closePrices[stock][datapoint]
                    ratio = sellPrice/buyPrice
                    datapoint+=1
                # while datapoint<(j+timeout) and stopLoss<ratio<takeProfit:
                #     sellPrice=closePrices[stock][datapoint]
                #     ratio = sellPrice/buyPrice
                #     datapoint+=1
                
                tradePercent+=ratio-fee
                totalTime+=time
        
    try:
        tradePercent=tradePercent/total
    except:
        pass

    averageTime=((totalTime/86400)/total)+1
    tradePerDay = (tradePercent**(1/averageTime))
    return [tradePerDay, total]
    # print(tradePercent)
    # print(tradePerDay)
partialAlgo = partial(algo, openPrices=openPrices, closePrices=closePrices, timestamps=timestamps)

In [45]:
#change
minimum=1.00
maximum=1.1
difference=0.02
changeList=[]
j=minimum
while j<maximum+difference:
    changeList.append(j)
    j+=difference

#takeProfit
minimum=1.05
maximum=1.20
difference=0.05
takeProfitList=[]
j=minimum
while j<maximum+difference:
    takeProfitList.append(j)
    j+=difference

#stopLoss
minimum=0.80
maximum=0.95
difference=0.05
stopLossList=[]
j=minimum
while j<maximum+difference:
    stopLossList.append(j)
    j+=difference

#timeout
timeoutList=[]
dayList=[1,2,3,4,5,10,20,30,60]
for day in dayList:
    timeoutList.append(day*86400)

#daysBefore
daysBeforeList=[1,2,3,4,5,10,20,30,60]

#minimum total for algo
minTotal=5000

#rough commission fee
fee=[0.01]

fullCombinations=list(product(changeList, takeProfitList, stopLossList, timeoutList, daysBeforeList, fee))
len(fullCombinations)

7776

In [12]:
algo(fullCombinations[0], openPrices, closePrices, timestamps)
fullCombinations[0]

(1.0, 1.05, 0.8, 86400, 1, 0.01)

In [13]:
%%timeit -n5
algo(fullCombinations[0], openPrices, closePrices, timestamps)

27.8 ms ± 512 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [14]:
%%timeit -n5
partialAlgo(fullCombinations[0])

29.2 ms ± 669 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [15]:
results=[]
with multiprocessing.Pool(20) as pool:
    for result in pool.map(
        partialAlgo,
        fullCombinations
    ):
        if result[1]>=minTotal:
            results.append(result[0])

In [36]:
sortedResults, resultCombinations = zip(*sorted(zip(results, fullCombinations), reverse=True))

In [37]:
sortedResults[0]

1.004674180322187

In [39]:
bestStrats=[]
for topResult in range(200):
    bestStrats.append([fullCombinations[topResult],partialAlgo(resultCombinations[topResult])])

In [40]:
bestStrats

[[(1.0, 1.05, 0.8, 86400, 1, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 2, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 3, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 4, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 5, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 10, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 20, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 30, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 86400, 60, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 1, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 2, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 3, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 4, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 5, 0.01), [1.004674180322187, 7595.0]],
 [(1.0, 1.05, 0.8, 172800, 10, 0.01), [1.004674180322187, 7595.0]],